In [38]:
import pandas as pd
import numpy as np

from keras.utils.data_utils import get_file


## Pull the raw data and look at a few examples

In [5]:
fname = 'europarl-test.zip'
floc = 'https://storage.googleapis.com/google-code-archive-downloads/v2/code.google.com/language-detection/'
fpath = get_file(fname,floc+fname)
print fpath

/home/ubuntu/.keras/datasets/europarl-test.zip


In [6]:
!unzip /home/ubuntu/.keras/datasets/europarl-test.zip

Archive:  /home/ubuntu/.keras/datasets/europarl-test.zip
  inflating: europarl.test           


In [37]:
with open('europarl.test', 'r') as f:
    rawtxt = f.readlines()

#Show a few raw examples
for t in rawtxt[:3]:
    print t

bg	"Европа 2020" не трябва да стартира нов конкурентен маратон и изход с приватизация.

bg	(CS) Най-голямата несправедливост на сегашната обща селскостопанска политика е фактът, че субсидиите се разпределят неравностойно и несправедливо между старите и новите държави-членки.

bg	(DE) Г-жо председател, г-н член на Комисията, по принцип съм против въвеждането на нови данъци.



### Convert to Numpy Arrays and Reshape to Our desired format

In [62]:
data = np.array([t.split('\t') for t in rawtxt])
data = np.hsplit(data, 2) 
labels = data[0].reshape(1,-1)[0]
text = data[1].reshape(1,-1)[0]

In [65]:
print labels.shape
print text.shape

(21000,)
(21000,)


## Todo:
- Load the vocab from training step, replace with integers, limit to 71 words
- Convert labels to 1 hot